In [1]:
import pandas as pd

In [2]:
!ls

 bib.py						   submission2.csv
 data_description.txt				   submission.csv
 full_train.py					   test.csv
'hourse v1.ipynb'				   train.csv
 house-prices-advanced-regression-techniques.csv   Untitled2.ipynb
 __pycache__					   Untitled3.ipynb
 sample_submission.csv				   Untitled.ipynb


In [3]:
data = pd.read_csv('train.csv')

In [17]:
data = pd.read_csv('train.csv')
data.fillna(0, inplace=True)
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,57.623288,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,34.664304,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,42.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,63.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [64]:
set(data.dtypes.values)

{dtype('int64'), dtype('float64'), dtype('O')}

In [75]:
def get_columns(types):
    columns=[]
    for t in types:
        cs = list(data.dtypes[data.dtypes == t].keys())
        columns = columns+cs
    return columns
#data.dtypes[data.dtypes == 'float64'].keys()
num_columns = get_columns(['float64', 'int64'])
num_columns.remove('Id')
num_columns.remove('SalePrice')


cat_columns = get_columns(['O'])

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attributes_names):
        self.attributes_names = attributes_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attributes_names]

In [96]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [97]:
num_pipeline = Pipeline([
    ("select_numeric", DataFrameSelector(numeric_columns)),
    ("imputer", SimpleImputer(strategy="median")),
    ("std_scaler", StandardScaler()),
])


In [30]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [79]:
cat_pipeline = Pipeline([
    ("select_cat", DataFrameSelector(cat_columns)),
    ("cat_encoder", OneHotEncoder(sparse=False))
])

In [35]:
from sklearn.pipeline import FeatureUnion

preprocess_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [84]:
X_train = preprocess_pipeline.fit_transform(data)

In [38]:
y_train = data['SalePrice']

In [ ]:
from sklearn.svm import SVC

svm_clf = SVC(gamma="scale", kernel='linear')
svm_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_train, svm_clf.predict(X_train))

In [ ]:
predict_data = pd.read_csv('test.csv')
predict_data[cat_columns] = 